In [1]:
###
# 指定した期における決算データでフィルタし、ランキングを出す

In [2]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import KessanPl, read_data
import polars as pl
from datetime import date

In [3]:
### 四半期データでやってみる
# 決算日の指定 -> 指定した日の四半期決算でランキングを作成する
target_settlement_date = date(2024, 9, 30)
# 売上高伸び率を何%以上にするか
min_sales_growthrate = 10

In [6]:
# 決算データの読み込み
fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL1 = KessanPl(df)
KPL2 = KessanPl(df)

# データ加工
KPL1.with_columns_growthrate_lastyear()
KPL2.with_columns_diff_growthrate()
KPL1.df = KPL1.df.filter(pl.col("settlement_type")=="四")
KPL2.df = KPL2.df.filter(pl.col("settlement_type")=="四")
df = KPL1.df.join(KPL2.df, on=["code", "settlement_date"], how="left")
df = df.select([
    "code", "settlement_date", "announcement_date", "sales", "operating_income", "sales_growthrate", "diff_operating_income_growthrate"
])
df = df.filter(pl.col("settlement_date")==target_settlement_date)\
    .filter(pl.col("operating_income")>0)\
    .filter(pl.col("sales_growthrate")>=min_sales_growthrate)\
    .filter(pl.col("diff_operating_income_growthrate")>0)
df = df.sort(
    by=[pl.col("diff_operating_income_growthrate")],
    descending=[True]
)

In [7]:
# diff_operating_income_growthrateが大きい順に並べ替えてランキング
df

code,settlement_date,announcement_date,sales,operating_income,sales_growthrate,diff_operating_income_growthrate
i64,date,date,i64,i64,f64,f64
4056,2024-09-30,2024-11-08,907,24,10.7,240.9
9130,2024-09-30,2024-11-01,3943,750,14.0,189.9
4502,2024-09-30,2024-10-31,1176038,184247,12.7,175.7
4177,2024-09-30,2024-11-13,1281,257,13.3,167.3
5254,2024-09-30,2024-11-14,762,362,10.8,145.9
…,…,…,…,…,…,…
7130,2024-09-30,2024-11-06,240006,3641,52.4,0.4
7214,2024-09-30,2024-11-14,27181,551,14.6,0.4
1446,2024-09-30,2024-11-14,3458,57,12.4,0.3
